In [1]:
#Necessary Imports
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import heapq

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#Necessary installs
!pip install -q xlrd
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 5.0 MB/s 
     |████████████████████████████████| 3.4 MB 31.4 MB/s 
     |████████████████████████████████| 6.8 MB 47.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.6 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=d3a224143b1d7424e729a96ece0ff5567ce1135bc51c5091dc6deb50455cdd5d
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
#Upload the Learning Unit.xlsx and Sample_Skills.xlsx
from google.colab import files
uploaded = files.upload()

Saving Learning Catalogue.xlsx to Learning Catalogue.xlsx
Saving Sample_Skills.xlsx to Sample_Skills.xlsx


In [5]:
#selecting model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [91]:
#Function to create sentence embeddings
#NAME = name for the xlsx file, DOCUMENT = name of the column with descriptions


def create_sentence_embeddings(NAME, DOCUMENT):
  df = pd.read_excel(NAME)
  df = pd.DataFrame(df, columns=[DOCUMENT])
  stop_words_l=stopwords.words('german')
  df['DOCUMENT_CLEANED']=df[DOCUMENT].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
  sentence_embeddings = model.encode(df.DOCUMENT_CLEANED)
  return sentence_embeddings
  return len(sentence_embeddings)

In [92]:
#Function for cosine similarity and saving a final notepad of learning catalogue and skills
#n= number of skills, embedding1= sentence embedding of the Learning Catalogue, embedding2= sentence embedding of the Sample Skills
#Skill_file= Sample_skills.xlsx, DOCUMENT = Skills, Learning_file= Learning_Catalogue.xlsx, DESCRIPTION= Description

cos = []
selected_skills = []
def highest_similarity_skills(n, embedding1, embedding2, Skill_file, DOCUMENT, Learning_file, DESCRIPTION):
  for i in range (len(embedding1)):
   cos_sim= cosine_similarity([embedding1[i]],embedding2)
   cos.append(cos_sim)
   
  highest_sim_index = []
  for i in range (len(cos)):
    for j in range (len(cos[i])):
      n_large=heapq.nlargest(n,range(len(cos[i][j])),np.array(cos[i][j]).__getitem__)
      highest_sim_index.append(n_large)


  highest_sim_index = np.array(highest_sim_index)

  for i in range (len(highest_sim_index)):
    skill = pd.read_excel(Skill_file)
    skill = pd.DataFrame(skill, columns=[DOCUMENT])
    highest_sim = skill.iloc[highest_sim_index[i]]
    selected_skills.append(highest_sim)
  
  learn = pd.read_excel(Learning_file)
  learn = pd.DataFrame(learn, columns=[DESCRIPTION])
   
  Final_file = learn.assign(skill=selected_skills)
  Final_file.to_csv('Selected skills for learning.csv')
  files.download('Selected skills for learning.csv')






In [93]:
Learning_embeddings = create_sentence_embeddings('Learning Catalogue.xlsx' , 'Description')

Skills_embeddings = create_sentence_embeddings('Sample_Skills.xlsx' , 'Skill')

In [94]:
highest_similarity_skills(5, Learning_embeddings , Skills_embeddings, 'Sample_Skills.xlsx' , 'Skill' , 'Learning Catalogue.xlsx' , 'Description')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>